importing library

In [1]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests

Make a GET request to fetch the raw HTML content


In [75]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text


# Step 2: Parse the html content
soup = BeautifulSoup(html_content)
#print(soup.prettify()) # print the parsed data of html



#Get the table having the class wikitable
table = soup.find("table", attrs={"class": "wikitable"})
table_data = table.tbody.find_all("tr")  # contains 2 rows
print ( len(table_data))






288


putiting to data frame

In [98]:
# Get all the headings 
headings = []
for th in table_data[0].find_all("th"):
    headings.append(th.text)

#tranforming to dictionary 
data={}
for i in range(1,len(table_data)): 
    content=[]
    for td in table_data[i].find_all("td"):
        content.append(td.text)
    data.setdefault(headings[0],[]).append(content[0])
    data.setdefault(headings[1],[]).append(content[1])
    data.setdefault(headings[2],[]).append(content[2])

# 
data=pd.DataFrame.from_dict(data)
# remove /n
data = data.replace('\n',' ', regex=True)

# delete rows with 'Not assigned' in the 'Borough' coloumn
indexNames = data[ data['Borough'] == 'Not assigned' ].index
data.drop(indexNames , inplace=True)

# repalce if Neighbourhood has 'Not assigned'
indexNames = data[ data['Neighbourhood\n'].str.strip() == 'Not assigned' ].index
data['Neighbourhood\n'][indexNames]=data['Borough'][indexNames]
#




combine two Neighbourhood with the same Postcode and Borough 
and make into separate columns

In [101]:

data1=data.groupby(['Postcode','Borough'],as_index = True)['Neighbourhood\n'].apply(lambda x: pd.Series(list(x))).unstack().reset_index()

data1.columns=['Postcode','Borough','Neighbourhood1','Neighbourhood2','Neighbourhood3','Neighbourhood4','Neighbourhood5','Neighbourhood6','Neighbourhood7','Neighbourhood8']
data1['Neighbourhoodall'] = data1[['Neighbourhood1', 'Neighbourhood2','Neighbourhood3','Neighbourhood4','Neighbourhood5','Neighbourhood6','Neighbourhood7','Neighbourhood8']].apply(lambda x: ','.join(x.fillna('').map(str)), axis=1)
data1['Neighbourhoodall'] = data1['Neighbourhoodall'].str.strip(',')
data1['Neighbourhoodall']
datawithcomma = data1[['Postcode','Borough','Neighbourhoodall']].copy()
datawithcomma.shape



(103, 3)